In [1]:
test_index = 0

#### testing

In [41]:
from load_data import *

In [42]:
# load_data()

## Loading the data

In [43]:
from load_data import *

In [44]:
X_train,X_test,y_train,y_test = load_data()

4
OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Shine/shine131.jpg
OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Rain/rain141.jpg


In [45]:
len(X_train),len(y_train)

(843, 843)

In [46]:
len(X_test),len(y_test)

(280, 280)

## Test Modelling

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [48]:
class Test_Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.c1 = nn.Conv2d(1,64,5)
        self.c2 = nn.Conv2d(64,128,5)
        self.c3 = nn.Conv2d(128,256,5)
        self.fc4 = nn.Linear(256*10*10,256)
        self.fc6 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,4)
    
    def forward(self,X):
        preds = F.max_pool2d(F.relu(self.c1(X)),(2,2))
        preds = F.max_pool2d(F.relu(self.c2(preds)),(2,2))
        preds = F.max_pool2d(F.relu(self.c3(preds)),(2,2))
#         print(preds.shape)
        preds = preds.view(-1,256*10*10)
        preds = F.relu(self.fc4(preds))
        preds = F.relu(self.fc6(preds))
        preds = self.fc5(preds)
        return preds

In [49]:
device = torch.device('cuda')

In [50]:
BATCH_SIZE = 32

In [51]:
IMG_SIZE = 112

In [52]:
model = Test_Model().to(device)

In [53]:
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [54]:
criterion = nn.CrossEntropyLoss()

In [55]:
EPOCHS = 12

In [56]:
from tqdm import tqdm

In [57]:
PROJECT_NAME = 'Weather-Clf'

In [58]:
import wandb

In [59]:
test_index += 1
wandb.init(project=PROJECT_NAME,name=f'test-{test_index}')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds.to(device)
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item()})
wandb.finish()

  0%|          | 0/12 [00:00<?, ?it/s]/home/indika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 12/12 [00:08<00:00,  1.42it/s]


loss,0.7543
_runtime,14
_timestamp,1621349033
_step,323


loss,██████▇▇█▇██▇█▆▆▆▆▅▆▅▇▆▆▅▅▄▄▅▅▄▄▄▅▄▁▃▃▆▁
_runtime,▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [60]:
for index in range(10):
    print(torch.argmax(preds[index]))
    print(y_batch[index])
    print('\n')

tensor(1, device='cuda:0')
tensor(0, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(1, device='cuda:0')
tensor(0, device='cuda:0')


tensor(3, device='cuda:0')
tensor(3, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(1, device='cuda:0')
tensor(0, device='cuda:0')


tensor(3, device='cuda:0')
tensor(3, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


